In [ ]:
#installation
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-12 03:38:13--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-04-12 03:38:13 (6.03 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SportsCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
#drop null

df = df.dropna()
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
#get count
df.count()

4849000

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#now we want to transform the DF to fit the review_id table

review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-08-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-08-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-08-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-08-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-08-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-08

In [ ]:
#drop the duplicates
review_id_df = review_id_df.dropDuplicates(["review_id"])

review_id_df

DataFrame[review_id: string, customer_id: string, product_id: string, product_parent: string, review_date: string]

In [ ]:
from pyspark.sql.types import * 

review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df["review_date"].cast(DateType()))

review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#now we want to transform the DF to fit the products table
product_id_df = df.select(["product_id", "product_title"])
product_id_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B012P7UPSM|Chicago Blackhawk...|
|B001GQ3VHG|Copag Poker Size ...|
|B008VS8M58|Baoer 223 5.56x45...|
|B005F06F4U|All Terrain Tackl...|
|B010T4IE2C|Swim Cap - 3 Pack...|
|B004RKJGLS|adidas Men's Spor...|
|B005V3DCBU|Nike Men's Perfor...|
|B00MHT9WN8|Green Bay Packers...|
|B001CSIRQ8|Isokinetics Inc. ...|
|B001KZ3NOO|Aottop Quality El...|
|B00005RCQS|Everlast 2'x6' Fo...|
|B00FA7RWVI|OGIO Men's Shredd...|
|B000MMH2HU|Blue Sea Systems ...|
|B00WJOATDS|Zengi AR15 45 Deg...|
|B00BOM2JNY|Black Mountain Pr...|
|B00OD5GAFM|Trijicon VCOG 1-6...|
|B00CAHDC1K|Naruto Headband f...|
|B004NLHXLG|Under Armour Men'...|
|B00L7OANWI|Yes4All Deep Tiss...|
|B0085PPSIQ|Lansky PS-MED01 B...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#drop duplicates
product_id_df = product_id_df.dropDuplicates(["product_id"])
product_id_df

DataFrame[product_id: string, product_title: string]

In [ ]:
product_id_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
#transform dataframe for customers table

customers_id_df = df.select(["customer_id"])
customers_id_df.show()

+-----------+
|customer_id|
+-----------+
|   48945260|
|    5782091|
|   45813853|
|    1593730|
|   29605511|
|   11112959|
|     108031|
|   13981540|
|   37993909|
|   26040213|
|   34657602|
|   14346192|
|   38782687|
|   27138575|
|   11838771|
|     535800|
|   23156579|
|   48107879|
|   27260960|
|   39537314|
+-----------+
only showing top 20 rows



In [ ]:
#use groupby for customer_id count

customers_id_df = customers_id_df.groupBy("customer_id").count()
customers_id_df.orderBy("customer_id").select(["customer_id", "count"])
customers_id_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   45193257|   21|
|   44934924|    1|
|   11614909|    1|
|   19186664|    1|
|   50492011|    1|
|   12988293|    3|
|   22848554|    4|
|   42126527|    1|
|   48782599|    1|
|   34146651|   10|
|   43194960|    3|
|   14127895|    1|
|   30636778|   44|
|   30274993|    1|
|   28069251|    1|
|   20545025|    5|
|   28029004|    4|
|   51290010|    1|
|   37367363|    1|
|    2583392|    7|
+-----------+-----+
only showing top 20 rows



In [ ]:
customers_id_df = customers_id_df.withColumn("customer_id",customers_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("count",customers_id_df["count"].cast(IntegerType()))

customers_id_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- count: integer (nullable = false)



In [ ]:
#transform to vine table
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|
|R32M0YEWV77XG8|          5|            1|          1|   N|
| RR8V7WR27NXJ5|          1|            0|          0|   N|
|R1MHO5V9Z932AY|          5|            0|          0|   N|
|R16PD71086BD2V|          5|            0|          1|   N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|
|R3AUMSHAW73HWN|          4|            2|          3|   N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|
|R3H9543FWBWFBU|          5|            1|          1|   N|
| RUANXOQ9W3OU5|          5|            0|          0|   N|
|R31673RTGEZSW7|          5|            2|          2|   N|
|R22OQLFSH42RCM|          5|            1|          1|   N|
|R12LEL4F3TSZUJ|          5|            2|          2|   N|
|R2L9XWD03072NI|          5|            

In [ ]:
#drop duplicates
vine_df = vine_df.dropDuplicates(["review_id"])

In [ ]:
#print schema
vine_df = vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_df["total_votes"].cast(IntegerType()))

vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
#code to write into dataFrame
mode = "append"
jdbc_url="[url]"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}



In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)
product_id_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)
customers_id_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)